In [3]:
from pycox.models import DeepHitSingle
from pycox.evaluation import EvalSurv
import torchtuples as tt

from tqdm import tqdm
import pandas as pd
import numpy as np
import torch

np.random.seed(42)
torch.manual_seed(42)

In [4]:
# base = '/data/processed_datasets/processed_datasets/mimic/phenotype/'
# phenotype = 'masld'
# model = 'motor'

# path = base + phenotype + '/' + model + '/features_with_label/'

base = '/user/zj2398/cache/motor_mimic_8k/embeddings/'
# phenotype_list = ['masld']
phenotype = 'stroke'
model = 'motor'
path = base + phenotype + '/' + model + '/features_with_label/'


In [5]:
# Use 10% for validation
train = pd.read_parquet(path + 'train.parquet')
val = train.sample(frac = 0.1, replace = False)
train = train.drop(val.index)
test = pd.read_parquet(path + 'test.parquet')

In [7]:
print(train)
# train['time_to_event_days'].max()/365

        subject_id     prediction_time  boolean_value  \
0         17947312 2154-08-28 13:30:00          False   
1         14066184 2140-10-07 01:15:00          False   
2         18237256 2117-03-01 11:00:00          False   
3         12330994 2177-06-27 14:04:00          False   
4         10538657 2149-03-12 21:55:00          False   
...            ...                 ...            ...   
281599    14244279 2151-02-06 07:17:41          False   
281600    19172342 2166-10-01 14:57:00          False   
281601    14686541 2122-08-16 16:08:00          False   
281602    15229574 2126-02-15 17:55:00          False   
281603    14313753 2204-08-24 09:11:00          False   

                                                 features  
0       [-1.1979039, 0.11037254, -0.70074356, -0.74835...  
1       [-0.94552845, -0.1191774, -0.58400804, 0.89763...  
2       [-1.0040412, -0.12654106, -0.682588, 0.3821438...  
3       [0.31768882, 1.2767233, -0.4727602, 0.18695708...  
4       [1.3246

In [8]:
train[~train.boolean_value].time_to_event_days.plot.hist(bins = 1000)

AttributeError: 'DataFrame' object has no attribute 'time_to_event_days'

In [ ]:
# Transform labels
num_durations = 100
labtrans = DeepHitSingle.label_transform(num_durations)
get_target = lambda df: (df['time_to_event_days'].values, df['boolean_value'].values) 

# Extract features and labels
x_train = np.stack(train.features.values)
y_train = labtrans.fit_transform(*get_target(train))
train = (x_train, y_train)

x_val = np.stack(val.features.values)
y_val = labtrans.fit_transform(*get_target(val))
val = (x_val, y_val)

x_test = np.stack(test.features.values)
durations_test, events_test = get_target(test)

In [ ]:
# Define one layer NN for Deephit
in_features = x_train.shape[1]
out_features = labtrans.out_features

net = tt.practical.MLPVanilla(in_features, [], out_features)

In [ ]:
# Train Deephit
model = DeepHitSingle(net, tt.optim.Adam, duration_index=labtrans.cuts)
epochs, batch = 100, 2048
log = model.fit(x_train, y_train, batch, epochs, [tt.callbacks.EarlyStopping()], val_data=val)

In [ ]:
# Check loss on the train and validation set
_ = log.plot()

In [ ]:
# Predict on test set
surv = model.predict_surv_df(x_test)

In [ ]:
# Compute boostrapped performance
results = {'C-Index':[], 'Int-Brier':[]}
time_grid = np.linspace(durations_test.min(), durations_test.max(), 100)
for boot in tqdm(range(100)):
    sample = np.random.choice(len(x_test), len(x_test))
    ev = EvalSurv(surv[sample], durations_test[sample], events_test[sample], censor_surv='km')
    results['C-Index'].append(ev.concordance_td('antolini'))
    results['Int-Brier'].append(ev.integrated_brier_score(time_grid))

In [ ]:
# Display
for metric in results:
    print('{}: {:.2f} ({:.2f})'.format(metric, np.mean(results[metric]), np.std(results[metric])))